In [4]:

# Step 1: Import libraries
import pandas as pd
import pingouin as pg

# Step 2: Load Excel file (adjust path if needed)
file_path = r #paste path here 
df = pd.read_excel(file_path, sheet_name="Sheet1")

# Step 3: Convert wide to long format
rater_blocks = ['A', 'B', 'C', 'D', 'E']
score_data = []

for i, rater in enumerate(rater_blocks):
    col_offset = i * 4
    desktop_scores = pd.to_numeric(df.iloc[1:, col_offset + 1], errors='coerce').reset_index(drop=True)
    oculus_scores = pd.to_numeric(df.iloc[1:, col_offset + 3], errors='coerce').reset_index(drop=True)
    sketch_ids = df.iloc[1:, col_offset].reset_index(drop=True)

    for condition, scores in zip(["Desktop", "Oculus"], [desktop_scores, oculus_scores]):
        temp_df = pd.DataFrame({
            'sketch_id': sketch_ids,
            'rater': rater,
            'condition': condition,
            'rating': scores
        })
        score_data.append(temp_df)

# Combine and clean
long_df = pd.concat(score_data, ignore_index=True)
long_df.dropna(subset=['rating'], inplace=True)

# Step 4: Compute ICC(3,1) for each condition
icc_results = {}

for condition in ['Desktop', 'Oculus']:
    df_condition = long_df[long_df['condition'] == condition]

    icc = pg.intraclass_corr(
        data=df_condition,
        targets='sketch_id',
        raters='rater',
        ratings='rating'
    )

    icc3 = icc[icc['Type'] == 'ICC3']
    icc_results[condition] = icc3

# Step 5: Display results
print("🔹 ICC(3,1) for Desktop condition:\n")
print(icc_results['Desktop'])
print("\n🔹 ICC(3,1) for Oculus condition:\n")
print(icc_results['Oculus'])


🔹 ICC(3,1) for Desktop condition:

   Type          Description       ICC          F  df1  df2          pval  \
2  ICC3  Single fixed raters  0.816573  23.258835   34  136  1.111216e-41   

          CI95%  
2  [0.72, 0.89]  

🔹 ICC(3,1) for Oculus condition:

   Type          Description       ICC          F  df1  df2          pval  \
2  ICC3  Single fixed raters  0.798226  20.780231   35  140  4.544693e-40   

         CI95%  
2  [0.7, 0.88]  
